<a href="https://colab.research.google.com/github/Moubarack-diop/Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importation des données

Dans cette étape, nous avons eu à importer le fichier CSV qui contient les données FAQs qu'on a extraites du site de Orange Assistance https://assistance.orange.sn/. Cette extraction s'est faite via du Web Scraping en utilisant Beautiful Soup.
Le fichier est constitué de deux colonnes: question et answer


In [ ]:
from google.colab import files
import pandas as pd

# Importer le fichier
uploaded = files.upload()
# Lire le fichier CSV en tant que DataFrame
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Contenu de {filename} :")
df.head()

## Importation des bibliothèques

In [ ]:
!pip install langchain chromadb transformers sentence-transformers bitsandbytes
!pip install -U langchain-community
!pip install langchain
!pip install torch
!pip install accelerate
!pip install python-telegram-bot

**Langchain** : C'est une bibliothèque Python utilisée pour construire des chaînes (chains) d'interactions avec des modèles de langage comme les LLM. Elle est particulièrement utile pour intégrer des bases de données et gérer les prompts.

**chromadb**: C'est une base de données vectorielle légère. Elle est utilisée pour stocker et rechercher des vecteurs (comme ceux générés à partir de texte), souvent dans des systèmes de récupération augmentée (RAG).

**transformers** :C'est une bibliothèque de Hugging Face pour utiliser des modèles de machine learning, comme GPT, BERT, ou Llama, dans des tâches de traitement du langage naturel.

**sentence-transformers** : Spécialisée dans la création d'encodages vectoriels pour des textes, elle est utilisée pour des tâches comme la recherche sémantique ou la détection de similarités.

**bitsandbytes** : Une bibliothèque optimisée pour les calculs de faible précision sur GPU, permettant d'exécuter de gros modèles LLM tout en consommant moins de mémoire.

**-U** : Cette option permet de mettre à jour le paquet à la dernière version disponible.

**langchain-community** : Fournit des intégrations, outils et fonctionnalités supplémentaires développés par la communauté autour de LangChain.

**torch** : Une bibliothèque de deep learning, très utilisée pour entraîner et exécuter des modèles de machine learning. De nombreux modèles LLM reposent sur PyTorch.

**accelerate** : Une bibliothèque de Hugging Face qui permet de gérer efficacement l'entraînement et l'inférence sur GPU ou CPU, notamment dans des environnements distribués.

**python-telegram-bot** : Une bibliothèque pour créer et gérer des bots Telegram en Python. Elle eprmet l'intégration de chatbot sur Télégram

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import transformers
from transformers import AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
import torch
from torch import cuda, bfloat16
from langchain.chains import RetrievalQA, LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from huggingface_hub import login
from langchain.document_loaders.csv_loader import CSVLoader

## Configuration de la quantification de la mémoire

In [ ]:
# Configuration pour la quantification du modèle
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Détecter si un GPU est disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

Dans l'implémentation d'un chatbot utilisant un LLM, l'intégration de la quantification de la mémoire est très importante. Les modèles de LLM sont très volumineux, et cela consomme beaucoup de mémoire GPU lors de leur chargement sur Google Colab. Afin d'optimiser notre mémoire GPU nous avons introduit la bibliothèque BitsAndBytes

## Connexion à Hugging Face

In [ ]:
# Connexion au Hugging Face Hub
login("HugginFace_Token")

## Chargement des données

Cette étape consiste à charger nos données contenues dans le fichier final_data.csv et de les transformer en DataFrame.
**Document**, qui est une classe Langchain va nous permettre ici de bien formater les documents charger.Cela facilite la recherche, l'interrogation des données.

In [ ]:
# Chargement du fichier CSV
df = pd.read_csv('final_data.csv')

In [ ]:
# Créer des documents à partir des questions et réponses
documents = [Document(page_content=row['question'], metadata={"answer": row['answer']}) for index, row in df.iterrows()]

## Text Splitting


Le text splitting consiste à diviser les textes contenus dans le document chargé en de plus petits élèments appelé chunks.

In [ ]:
# Découper les documents en chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

**RecursiveCharacterTextSplitter**: C'est une classe de LangChainutiliser pour découper des textes de manière intelligente.
Contrairement à un découpage strict basé sur une taille fixe, il divise le texte en respectant les limites naturelles comme les mots, les phrases ou les paragraphes pour éviter de couper au milieu d'une idée ou d'une phrase.

**chunk_size=1000** : Cela spécifie que chaque segment (chunk) doit contenir au maximum 1 000 caractères.
C'est utile pour respecter les limites des modèles ou faciliter la manipulation des segments.

**chunk_overlap=20** : Indique que chaque segment peut chevaucher le précédent sur 20 caractères.
Ce chevauchement garantit que les informations importantes à la frontière entre deux segments ne sont pas perdues, améliorant la continuité contextuelle.

**split_documents(documents)**: documents est une liste d'objets contenant les données textuelles à découper.
Chaque document de la liste est découpé en plusieurs segments (chunks) selon les règles définies par chunk_size et chunk_overlap.

**all_splits**: C'est une liste qui contient tous les segments générés après le découpage des documents.


## Embedding

Après avoir chargé et découpé les documents, les chunks seront transformés en données vectorielles. Pour cela nous avons utilisé
 le modèle d'embedding de Hugging Face  **sentence-transformers/all-mpnet-base-v2**

In [ ]:
# Charger le modèle d'embeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": device})

## Création de la base de données vectorielles avec ChromaDB



ChromaDB est un Vector Store qui va nous permettre de stocker les données vectorielles (data embedding)


In [ ]:
# Créer une base de données vectorielle avec Chroma
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

## Chargement du modèle et du tokeniser

Nous allons maintenant charger notre modèle de langage préentrainé,ainsi que son tokeniser.

**model_id**: est une chaîne qui spécifie l'identifiant du modèle à charger.Dans notre cas, nous avons utilisé Llama 2 13B Chat, un modèle de langage développé par Meta AI, disponible via l'interface de Hugging Face Transformers.

**AutoModelForCausalLM** : Cette classe est utilisée pour charger des modèles conçus pour des tâches de génération de texte (causal language modeling). Les modèles causaux prédisent le mot suivant dans une séquence donnée.

**from_pretrained** : Cette méthode télécharge et charge le modèle pré-entraîné spécifié par model_id. Cela inclut le téléchargement des poids du modèle depuis les serveurs de Hugging Face.

**quantization_config** : (optionnel) Définit la configuration de quantification,afin d'optimiser la mémoire et la vitesse d'exécution via bnb_config.

**torch_dtype** : Spécifie le type de données pour les poids du modèle.
torch.float16 (16 bits) est utilisé pour exécuter le modèle en virgule flottante 16 bits, réduisant ainsi les besoins en mémoire et accélérant l'exécution sur les GPU.

**torch.float32 (32 bits)** est utilisé comme solution de secours sur les CPU, car ils ne supportent généralement pas la virgule flottante 16 bits.

**device** : Vérifie si CUDA (GPU) est disponible pour effectuer le calcul. Si device == "cuda", la précision de 16 bits est utilisée.

**AutoTokenizer** : C'est une classe générique qui charge le tokenizer correspondant au modèle spécifié.

**Le tokenizer** est responsable de convertir du texte brut en tokens (unités compréhensibles par le modèle) et de reconstruire le texte à partir des prédictions du modèle.


In [ ]:
# Charger le modèle et tokenizer
model_id = 'meta-llama/Llama-2-13b-chat-hf'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Initialisation de la pipeline

In [ ]:
# Créer le pipeline de génération
query_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)


On configure un pipeline de génération de texte en utilisant la bibliothèque Hugging Face Transformers. Le pipeline combine notre modèle `Llama` pré-entraîné (`model`) et son tokenizer (`tokenizer`), qui encode et décode les données textuelles. Il optimise également les performances en définissant le type de données à utiliser : **`float16`** pour des calculs rapides et économes en mémoire sur GPU, ou **`float32`** pour des calculs précis sur CPU. Une fois configuré, ce pipeline sera appelé dans la suite pour produire du texte généré en fonction des requêtes ou des prompts spécifiques.

In [ ]:
# Initialiser la mémoire
memory = ConversationBufferWindowMemory(
    k=5,  # Garder la dernière conversation
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)


On initialise un système de mémoire conversationnelle en utilisant une classe appelée **`ConversationBufferWindowMemory`**, qui est conçue pour conserver un historique des interactions entre l'utilisateur et le Chatbot. La mémoire est configurée pour stocker uniquement les **5 dernières interactions** (grâce au paramètre `k=5`), ce qui permet de maintenir un historique limité et pertinent pour les réponses contextuelles. Elle associe les clés d'entrée et de sortie : la **question** (`input_key="question"`) et la **réponse** (`output_key="answer"`), facilitant ainsi le suivi des échanges. En activant **`return_messages=True`**, les messages sont retournés sous une forme structurée, rendant l'historique plus accessible pour une utilisation dans la suite du programme.

In [ ]:
def generate_answer(query):
    """
    Génère une réponse à la question de l'utilisateur en tenant compte de l'historique
    """
    try:
        torch.cuda.empty_cache()
        # Récupérer des documents pertinents
        docs = vectordb.similarity_search(query)

        # Récupérer l'historique de la conversation
        chat_history = memory.load_memory_variables({})["chat_history"]

        # Formater l'historique des conversations
        formatted_history = ""
        if chat_history:
            formatted_history = "\n".join([
                f"{'Utilisateur' if message.type == 'human' else 'Assistant'}: {message.content}"
                for message in chat_history
            ])

        # Vérifier si des documents ont été trouvés
        if docs:
            all_retrieved_answers = "\n".join([doc.metadata['answer'] for doc in docs])
        else:
            return "Je suis désolé, je n'ai pas assez d'informations pour répondre à cette question."

        # Préparer le contexte complet avec l'historique
        full_context = f"""
Tu es Tontoo, une Intelligence Artificielle dédiée à fournir des réponses sur Orange Sénégal.
Ton rôle est de transmettre des informations utiles et pertinentes en te basant strictement sur le contexte fourni.

Directives :

1. Réponse basée sur le contexte fourni :
   - Utilise exclusivement les informations dans le contexte donné pour répondre aux questions.
   - Si une information est absente du contexte ou si tu n’as pas la réponse, dis : « Je n'ai pas assez d'informations. Consulte www.orange.sn pour plus de détails. »
   - Inclus impérativement tous les liens disponibles dans le contexte.

2. Gestion du contexte :
   - Si le contexte est vide, réponds simplement : « Je n'ai aucune information. Consulte www.orange.sn pour plus de détails. »
   - Ne fais jamais d’hypothèses ou de suppositions en l'absence de contexte.

3. Traitement des questions et du langage :
   - Si la question contient des propos inappropriés ou offensants, réponds : « Je ne réponds pas à ce type de langage. »
   - Si la demande est vague ou ambiguë, invite l’utilisateur à préciser sa question pour mieux répondre.

4. Style de réponse :
   - Ne réponds à aucune question qui n'est pas en rapport avec Orange Sénégal.
   - Réponds de manière concise, professionnelle et amicale.
   - Évite de montrer les documents de référence ; concentre-toi uniquement sur les réponses claires et directes.

5. Exactitude et transparence :
   - Ne fais pas de spéculations et sois précis dans tes réponses.
   - Se limiter imperativement a répondre a la question posée
   - Si une information te semble incertaine ou ambiguë, indique-le clairement.
   - évite au maximum les répétitions dans tes réponses

6. Spécificité à Orange Sénégal :
   - Mets en avant les offres, produits et services spécifiques à Orange Sénégal.
   - Rappelle aux utilisateurs de visiter www.orange.sn pour obtenir des informations complètes et actualisées.


Documents pertinents trouvés:
{all_retrieved_answers}

Historique des conversations:
{formatted_history}

Question actuelle: {query}
Réponse:"""

        # Générer la réponse
        outputs = query_pipeline(
            full_context,
            max_new_tokens=1500,
            clean_up_tokenization_spaces=True
        )
        response = outputs[0]["generated_text"].split("Réponse:")[-1].strip()

        # Sauvegarder le contexte dans la mémoire
        memory.save_context(
            {"question": query},
            {"answer": response}
        )

        return response

    except Exception as e:

        print(f"Error: {e}")
        torch.cuda.empty_cache()
        return "Désolé, je n'ai pas pu traiter votre demande."

def clear_memory():
    """Efface l'historique de la conversation"""
    memory.clear()

def get_conversation_history():
    """Récupère l'historique de la conversation"""
    return memory.load_memory_variables({})["chat_history"]

def print_conversation_history():
    """Affiche l'historique de la conversation"""
    history = get_conversation_history()
    if not history:
        print("Aucun historique de conversation disponible.")
        return

    print("\n=== Historique des conversations ===")
    for message in history:
        role = "Utilisateur" if message.type == "human" else "Assistant"
        print(f"{role}: {message.content}\n")

Nous mettons en place un système conversationnel. Cela génère des réponses pertinentes en utilisant des documents extraits de notre base vectorielle, l’historique des interactions, et des directives spécifiques. La fonction principale, **`generate_answer(query)`**, commence par libérer la mémoire GPU pour optimiser les performances, puis recherche des documents pertinents liés à la requête de l'utillisateur dans la base de données vectorielle (**`vectordb.similarity_search`**). Elle récupère également l’historique des conversations via la mémoire (**`memory.load_memory_variables`**) pour fournir des réponses contextualisées. Si des documents pertinents sont trouvés, ils enrichissent le contexte, sinon un message standard informe l’utilisateur d’un manque d’informations. Un contexte global est ensuite construit en intégrant les documents, l’historique formaté, et des instructions strictes pour limiter les réponses au cadre d’**Orange Sénégal**. Ce contexte est traité par un pipeline de génération de texte (**`query_pipeline`**), qui produit une réponse nettoyée et concise, sauvegardée ensuite dans la mémoire pour des interactions ultérieures. Des fonctions auxiliaires, comme **`clear_memory`**, **`get_conversation_history`**, et **`print_conversation_history`**, permettent de gérer et d’afficher l’historique des conversations.

Historique des conversations:
{formatted_history}


In [ ]:
from telegram import Update, Bot
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import nest_asyncio

# Initialiser nest_asyncio pour Colab
nest_asyncio.apply()

# Configuration du token Telegram
TELEGRAM_TOKEN = 'Telegram_token'

# Fonction pour démarrer le bot avec un message d'accueil personnalisé pour Orange Sénégal
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    # Message d'accueil
    message = (
        "<b>Bonjour et bienvenue sur Tontoo, le Chatbot d'Orange Sénégal !</b>\n\n"
        "Ce chatbot est conçu pour vous offrir une assistance en temps réel. Posez vos questions et nous vous fournirons des réponses "
        "rapides et précises. Il comprend et répond en francais.\n\n"
        "Voici quelques commandes pour vous aider à naviguer :\n"
        "• /start - Démarrer le bot\n"
        "• /clear - Effacer l'historique\n"
        "• /historique - Afficher l'historique de la conversation\n\n"
        "Nous sommes là pour répondre à toutes vos questions. N'hésitez pas à interagir avec nous ! 😊"
    )
    # Envoyer le message
    await update.message.reply_text(message, parse_mode="HTML")


# Fonction pour traiter les messages utilisateur
async def answer(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    user_query = update.message.text
    response = generate_answer(user_query)
    torch.cuda.empty_cache()
    await update.message.reply_text(response)

# Fonction pour effacer la mémoire de conversation
async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    clear_memory()  # Appel de la fonction clear_memory pour effacer l'historique
    await update.message.reply_text("L'historique de la conversation a été effacé.")

# Fonction pour afficher la mémoire de conversation
async def historique(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    history = get_conversation_history()
    if not history:
        await update.message.reply_text("Aucun historique de conversation disponible.")
    else:
        formatted_history = "\n".join(
            f"{'Utilisateur' if message.type == 'human' else 'Assistant'}: {message.content}"
            for message in history
        )
        await update.message.reply_text(f"Historique des conversations :\n{formatted_history}")

# Créer une application Telegram
app = Application.builder().token(TELEGRAM_TOKEN).build()

# Ajouter les handlers pour les commandes et les messages
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("clear", clear))
app.add_handler(CommandHandler("historique", historique))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, answer))

# Démarrer le bot en mode polling
print("Bot en ligne sur Telegram.")
app.run_polling()

On intègre le bot sur Telegram en utilisant les bibliothèques **`telegram`** et **`telegram.ext`**. Le bot est configuré avec un **token Telegram** unique pour se connecter à l'application et fonctionne dans un environnement asynchrone comme Google Colab grâce à **`nest_asyncio`**. Plusieurs fonctionnalités sont proposées, notamment une commande **`/start`**, qui affiche un message d'accueil détaillé en HTML, présentant le chatbot et ses capacités, ainsi que les commandes disponibles. La commande **`/clear`** permet d'effacer l’historique des conversations via la fonction **`clear_memory`**, tandis que **`/historique`** affiche les échanges précédents sous un format lisible en récupérant les messages enregistrés dans la mémoire. Lorsqu’un utilisateur envoie un message texte, la fonction **`answer`** génère une réponse pertinente en appelant la fonction **`generate_answer`**. Les interactions sont gérées à l’aide de *handlers*, qui associent des commandes spécifiques ou des messages texte à leurs fonctions correspondantes. Enfin, le bot est lancé en mode *polling*, ce qui lui permet de surveiller en continu les nouveaux messages pour y répondre.